# Exercise Session 1: Kinematics & Dynamics - Pinocchio Tutorial

Running the code:
- Follow the installation instructions shared for *exercise_0_software_environment*
- Use the created software environment (`conda activate py13roboticscourse`)

In [1]:
import numpy as np
import pinocchio as pin
from pinocchio.utils import *

## Basics

In [2]:
qu = pin.Quaternion(0.7,0.2,0.2,0.6) # geometry
R = qu.matrix()
p = np.array([1,2,3])
T=pin.SE3(R,p) # transformation aMb=pin.SE3(aRb,apb)

print("R,p: ", T)
print("T: ", T.homogeneous) # homegeneous matrix
print("p: ", T.translation) # translation vector
print("R: ", T.rotation) # rotation matrix

T = pin.SE3(rotate('z',np.pi/4), np.array([0.1,0.05,0.3]))

print("T: ", T.homogeneous)

R,p:    R =
  0.2 -0.76  0.52
 0.92   0.2 -0.04
-0.04  0.52  0.84
  p = 1 2 3

T:  [[ 0.2  -0.76  0.52  1.  ]
 [ 0.92  0.2  -0.04  2.  ]
 [-0.04  0.52  0.84  3.  ]
 [ 0.    0.    0.    1.  ]]
p:  [1. 2. 3.]
R:  [[ 0.2  -0.76  0.52]
 [ 0.92  0.2  -0.04]
 [-0.04  0.52  0.84]]
T:  [[ 0.70710678 -0.70710678  0.          0.1       ]
 [ 0.70710678  0.70710678  0.          0.05      ]
 [ 0.          0.          1.          0.3       ]
 [ 0.          0.          0.          1.        ]]


## Loading and using a robot model (URDF), robot kinematics

In [3]:
"""
This code is simulating a robot in a virtual environment using Pinocchio, 
a robotics library for kinematics and dynamics. 
It helps understand how the robot moves and how its different parts 
(joints, collision models, and visual models) are positioned in space.

The code samples a random robot pose, computes forward kinematics, 
and updates the placements of joints, frames, collision objects, 
and visual objects. 
Finally, it prints the placements of all these elements 
in the world frame.
"""

urdf_model_path = "ur10_robot.urdf"
mesh_dir = "meshes"

# Load the urdf model
"""
URDF = Unified Robotics Description Format
The URDF file describes the robot's structure, including its:
Links (rigid parts like arms, hands, etc.)
Joints (how the links move relative to each other)
Collision model (used for detecting collisions)
Visual model (for rendering in a simulator)
"""
model, collision_model, visual_model = pin.buildModelsFromUrdf(
    urdf_model_path, mesh_dir
)
"""
model: The kinematic and dynamic model.
collision_model: The collision representation for collision checking.
visual_model: The visual representation for rendering.
"""
print("model name: " + model.name)



# Create data required by the algorithms
data, collision_data, visual_data = pin.createDatas(
    model, collision_model, visual_model
)
"""
data: Stores intermediate computations for 
forward/inverse dynamics, kinematics, etc.
collision_data: Stores collision-related data.
visual_data: Stores visualization-related data.
"""



"""
The goal is to:
Load the robot model: 
    Read the URDF file and create models for kinematics, collision, and visualization.
Generate a random configuration: 
    Assign random values to the robot's joint angles 
    (e.g., the arm moves to a random pose).
Compute forward kinematics: 
    Determine where each part of the robot is in 3D space.
Extract the position of the end-effector (gripper): 
    Find the transformation from the world frame to the end-effector 
    (the tool at the robot’s end, e.g., a hand or a welding tool).
Update the geometry: 
    Compute where the collision and visual objects should be in 3D space.
Print everything: 
    Display the positions of all joints, collision objects, and visual elements.
"""




# Sample a random configuration
# Generates a random joint configuration q
# within the robot's joint limits.
q = pin.randomConfiguration(model)
print(f"q: {q.T}")

# Perform the forward kinematics over the kinematic tree
"""
Computes the pose (position and orientation) of all joints and frames in the kinematic tree.
Updates the frame placements based on the computed forward kinematics.
"""
pin.forwardKinematics(model, data, q)
pin.updateFramePlacements(model, data)

# Get the transformation matrix from the world to the frame
# Retrieves the transformation matrix from the world 
# to the end-effector ("ee_link") frame.
frame_name = "ee_link"
T_pin = data.oMf[model.getFrameId(frame_name)]
print("\nT_{world,ee_link}:")
print(T_pin)

# Update Geometry models
# Updates the geometrical representations (collision and visual) 
# based on the computed kinematics.
pin.updateGeometryPlacements(model, data, collision_model, collision_data)
pin.updateGeometryPlacements(model, data, visual_model, visual_data)

# Print out the placement of each joint of the kinematic tree
# Iterates through all joints and prints their positions in the world frame.
print("\nJoint placements:")
for name, oMi in zip(model.names, data.oMi):
    print("{:<24} : {: .2f} {: .2f} {: .2f}".format(name, *oMi.translation.T.flat))

# Print out the placement of each collision geometry object
# Prints the world-frame positions of all collision objects.
print("\nCollision object placements:")
for k, oMg in enumerate(collision_data.oMg):
    print("{:d} : {: .2f} {: .2f} {: .2f}".format(k, *oMg.translation.T.flat))

# Print out the placement of each visual geometry object
# Prints the world-frame positions of all visual objects.
print("\nVisual object placements:")
for k, oMg in enumerate(visual_data.oMg):
    print("{:d} : {: .2f} {: .2f} {: .2f}".format(k, *oMg.translation.T.flat))

model name: ur10
q: [-0.28403702  1.6194398  -0.84958421  0.1684008   5.68288633  5.23006147]

T_{world,ee_link}:
  R =
-0.0893396   0.982843   0.161363
  0.885691   0.004287   0.464256
  0.455599   0.184394  -0.870878
  p = 0.314016 0.158369 -0.90871


Joint placements:
universe                 :  0.00  0.00  0.00
shoulder_pan_joint       :  0.00  0.00  0.13
shoulder_lift_joint      :  0.06  0.21  0.13
elbow_joint              : -0.01  0.06 -0.48
wrist_1_joint            :  0.38 -0.06 -0.88
wrist_2_joint            :  0.41  0.05 -0.88
wrist_3_joint            :  0.32  0.08 -0.95

Collision object placements:
0 :  0.00  0.00  0.00
1 :  0.00  0.00  0.13
2 :  0.06  0.21  0.13
3 : -0.01  0.06 -0.48
4 :  0.38 -0.06 -0.88
5 :  0.41  0.05 -0.88
6 :  0.32  0.08 -0.95
7 :  0.31  0.15 -0.91

Visual object placements:
0 :  0.00  0.00  0.00
1 :  0.00  0.00  0.13
2 :  0.06  0.21  0.13
3 : -0.01  0.06 -0.48
4 :  0.38 -0.06 -0.88
5 :  0.41  0.05 -0.88
6 :  0.32  0.08 -0.95


## Visualize

In [4]:
from pinocchio.visualize import MeshcatVisualizer as PMV
# Meshcat, a browser-based 3D visualization tool.

# This creates a visualizer (viz) that can display the robot model (kinematics, collision, and visual representations).
viz = PMV(model, collision_model, visual_model, collision_data=collision_data, visual_data=visual_data)
# The collision_data and visual_data are passed to allow visualization of both collision shapes and rendered model.
viz.initViewer(open=False) # initialize, do not open the viewer automatically

viz.loadViewerModel() # Load the robot model
# Display a robot configuration.
viz.display(q) # display random configuration
viz.displayVisuals(True) # Enable rendering of the robot's visual model
viz.displayFrames(True) # Displays coordinate frames
viz.updateFrames() # Update the coordinate frames so they reflect the latest robot configuration

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [19]:
# just some random configuration displaying
q = pin.randomConfiguration(model)
print(f"q: {q.T}")
viz.display(q)
viz.displayVisuals(True)
viz.displayFrames(True)
viz.updateFrames()

q: [-1.61371038  6.03655139 -0.52225353  1.86918142  0.77212906 -2.51547724]


## Dynamics

In [18]:
"""
This code performs dynamics computations for a robot using Pinocchio:
Forward dynamics (FD): Computes joint accelerations (ddq) given joint positions (q), velocities (v), and torques (tau).
Computing derivatives of the FD: Determines how joint accelerations change with respect to position, velocity, and torque.
Inverse dynamics (ID): Computes joint torques (tau) required to achieve a given motion using the Recursive Newton-Euler Algorithm (RNEA).

Forward Dynamics (Aba (Articulated Body Algorithm) Algorithm)
Goal: Given joint position, velocity, and torques, compute joint accelerations.
"""
v = np.random.rand(model.nv, 1)  # random joint velocity
tau = np.random.rand(model.nv, 1)  # random joint torques

# Forward dynamics (joint accelerations ddq)
ddq = pin.aba(model, data, q, v, tau)
print(f"ddq: {ddq}")

# Evaluate the derivatives
# Compute how joint accelerations change with respect to q, v, and tau.
pin.computeABADerivatives(model, data, q, v, tau)

# Retrieve the derivatives in data
ddq_dq = data.ddq_dq  # Derivatives of the FD w.r.t. the joint config vector, How acceleration changes with position
ddq_dv = data.ddq_dv  # Derivatives of the FD w.r.t. the joint velocity vector, How acceleration changes with velocity
ddq_dtau = data.Minv  # Derivatives of the FD w.r.t. the joint acceleration vector, How acceleration changes with torque
"""
ddq_dq: How changing joint positions affects joint accelerations.
ddq_dv: How changing joint velocities affects joint accelerations.
ddq_dtau: Inverse of the mass matrix, linking torque to acceleration.
"""

print(f"ddq_dq: {ddq_dq}")
print(f"ddq_dv: {ddq_dv}")
print(f"ddq_dtau: {ddq_dtau}")

# Inverse dynamics (joint torques), using RNEA
# Compute the joint torques (tau) needed to achieve a given motion.
tau = pin.rnea(model, data, q, v, ddq)
print(f"tau_est: {tau}")

"""
Forward dynamics (ABA): Given torque (tau), find acceleration (ddq).
Inverse dynamics (RNEA): Given acceleration (ddq), find torque (tau).
Derivative computations help in optimization, motion planning, and control.
"""

ddq: [ 1.00897350e+00 -1.06646314e+01  3.99540055e+00  7.92635598e+01
  1.25760059e+02  1.50727752e+03]
ddq_dq: [[-0.00000000e+00 -2.38395365e-01 -6.72349775e-01  1.24860900e-01
  -4.65978748e-03 -1.02069410e-05]
 [-0.00000000e+00 -1.83101632e+01  7.89465322e+00 -1.66816050e-01
  -9.06725323e-02 -3.15962156e-04]
 [-0.00000000e+00  3.23755308e+01 -2.82378611e+01  8.00579032e-01
  -5.31115763e-02 -1.80705222e-04]
 [-0.00000000e+00 -6.98004409e+00  2.70910589e+01  6.15575165e+00
  -5.60131721e+01 -1.94102913e-01]
 [-0.00000000e+00  1.28616806e+00  1.69978215e+00  9.40952409e-01
   8.60079476e-02 -8.62427567e-01]
 [-0.00000000e+00  3.28940730e+00  3.03882542e+00  3.28052732e+00
  -8.82342943e+01 -3.42660040e-01]]
ddq_dv: [[-1.13374235e+00 -3.48653809e-01 -9.38959213e-02  2.15295604e-03
   4.11861249e-04 -1.07924223e-06]
 [ 1.18675357e+00  1.51940829e+00  8.30641994e-01 -7.80009900e-03
  -3.11454247e-04 -3.34086088e-05]
 [-1.96468213e+00 -3.94985812e+00 -1.70301886e+00  4.00837792e-02
   1.

## CasADi-Pinocchio interface

In [7]:
from pinocchio import casadi as cpin
import casadi as ca
import os

Use cpin instead of pin for every function from Pinocchio

In [8]:
"""
The following code block do the following:
This code symbolically formulates and computes robot dynamics (forward and inverse dynamics, as well as forward kinematics) using CasADi and Pinocchio. 
The idea is to:
- Define symbolic representations of the robot’s joint states (position, velocity, torque, acceleration).
- Compute forward and inverse dynamics symbolically using Pinocchio.
- Convert these symbolic expressions into CasADi functions for efficient evaluation.
- Save and reload the functions, enabling their use in optimization, control, or simulation.

CasADi allows efficient computation of robot dynamics.
Symbolic expressions enable fast optimization, useful in trajectory planning and control.
It supports automatic differentiation, crucial for gradient-based optimization.
"""

# Define the equivalent casadi model and data
cmodel = cpin.Model(model)
cdata = cmodel.createData()

In [9]:
# Define CasADi symbols for joint positions (q), velocities (dq), and torques (tau)
cq = ca.SX.sym('q', model.nq)  # Joint positions (symbolic)
cdq = ca.SX.sym('dq', model.nq)  # Joint velocities (symbolic)
ctau = ca.SX.sym('tau', model.nq)  # Joint torques (symbolic)
"""
cq, cdq, and ctau are symbolic representations of the robot's joint positions, velocities, and torques.
This allows us to define symbolic functions that can later be evaluated efficiently.
"""


"""
Key Idea:
Articulated Body Algorithm (ABA) computes joint accelerations (ddq) given:
q (joint position)
dq (joint velocity)
tau (joint torque)
We convert this symbolic expression into a CasADi function and save it.
The function can be reused in optimization, control, or simulation, avoiding redundant computations.
"""
# Compute the forward dynamics
cddq = cpin.aba(cmodel, cdata, cq, cdq, ctau)
# Convert to casadi function
forward_dynamics_function = ca.Function('forward_dynamics', [cq, cdq, ctau], [ca.cse(cddq)], ['q', 'dq', 'tau'], ['ddq'])
# Save the function
os.makedirs('casadi_functions', exist_ok=True)
forward_dynamics_function.save('casadi_functions/forward_dynamics.casadi')

In [10]:
# do the same for inverse dynamics

# Define casadi symbols for the joints
cq = ca.SX.sym('q', model.nq)
cdq = ca.SX.sym('dq', model.nq)
cddq = ca.SX.sym('ddq', model.nq)

# Compute the inverse dynamics
ctau = cpin.rnea(cmodel, cdata, cq, cdq, cddq)
# Convert to casadi function
inverse_dynamics_function = ca.Function('inverse_dynamics', [cq, cdq, cddq], [
                                        ca.cse(ctau)], ['q', 'dq', 'ddq'], ['tau'])
# Save the function
inverse_dynamics_function.save(
    'casadi_functions/inverse_dynamics.casadi')

In [11]:
# do the same for forward kinematics

# Define casadi symbols for the joints
cq = ca.SX.sym('q', model.nq)
cdq = ca.SX.sym('dq', model.nq)
cddq = ca.SX.sym('ddq', model.nq)

# Compute the forwards kinematics
cpin.framesForwardKinematics(cmodel, cdata, cq)

frame_name = "ee_link"

# Get the transformation matrix from the world to the frame
T_pin = cdata.oMf[model.getFrameId(frame_name)]

# Combine the rotation and translation into one matrix
T = ca.SX.zeros(4, 4)
T[:3, :3] = T_pin.rotation
T[:3, 3] = T_pin.translation
T[3, 3] = 1

# Simplify the matrix
T = ca.cse(ca.sparsify(T, 1e-10))

# Define the casadi function
T_fk = ca.Function('T_fk', [cq], [T], ['q'], ['T_world_'+frame_name])

# Save the function
T_fk.save('casadi_functions/T_world_'+frame_name+'.casadi')

Load functions to reuse them without Pinocchio

In [12]:
"""
now we can Load functions to reuse them without Pinocchio
for example:
"""

# Forward dynamics
fwd_dyn = ca.Function.load('casadi_functions/forward_dynamics.casadi')

ddq = fwd_dyn(q, v, tau)
print(f"ddq: {ddq}")

"""
Why do this (use casadi)?
Symbolic Representation: Instead of computing forward/inverse dynamics numerically, we symbolically express them using CasADi.
Optimization & Control: These functions can be used for trajectory optimization, inverse kinematics, or model predictive control (MPC).
Efficiency & Reusability: The saved functions can be loaded and evaluated efficiently in real-time for robot motion control.
Automatic Differentiation: Since CasADi supports automatic differentiation, these functions can be differentiated for optimization problems.
"""

ddq: [-5.86468, -19.881, 4.08795, 25.199, 110.876, 895.486]
